In [ ]:
#Reference https://rumn.medium.com/part-1-ultimate-guide-to-fine-tuning-in-pytorch-pre-trained-model-and-its-configuration-8990194b71e

In [ ]:
!pip install wandb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split
import wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/CS6910-A2/CS6910-A2/train'
test_dir = '/content/drive/MyDrive/CS6910-A2/CS6910-A2/val'

#train_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'
#test_dir = '/kaggle/input/inaturalist/inaturalist_12K/val'
IMG_SIZE = (224,224)

# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),        # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_dir, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Mounted at /content/drive
cuda


In [ ]:
class pretrainedCNN():
  def __init__(self, trainset, valset, model, batch_size=32, freeze_percent = 1):
    # freeze earlier layers
    self.model = model
    if freeze_percent==1:
      for param in self.model.parameters():
        param.requires_grad = False
    else:
      total_layers = sum(1 for _ in model.parameters())
      num_layers_to_freeze = int(freeze_percent * total_layers)

      # Counter to track the number of frozen layers
      frozen_layers = 0

      # Freeze layers until reaching the desired percentage
      for param in self.model.parameters():
          param.requires_grad = False
          frozen_layers += 1
          if frozen_layers >= num_layers_to_freeze:
              break

    # Modify fully connected layer for fine-tuning
    num_classes = 10  # Number of classes in iNaturalist dataset
    num_ftrs = self.model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)
    for param in self.model.fc.parameters():
      param.requires_grad = True
    # DataLoader for the dataset
    self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True)
    self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle=True)

  def train(self, epochs = 10, lr=0.001, weight_decay=0):
    self.model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(1,epochs+1):
      train_accuracy = 0
      count = 0
      self.model.train()
      for inputs, labels in self.dataloader_train:
          # forward, backward, and then weight update
          inputs = inputs.to(device)
          labels = labels.to(device)
          # Forward pass
          optimizer.zero_grad()
          y_pred = self.model(inputs)
          loss = criterion(y_pred, labels)
          train_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
          count += len(labels)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      train_accuracy /= count
      print("Epoch %d: model train accuracy %.2f%%" % (epoch, train_accuracy*100))
      wandb.log({ 'epoch': epoch, 'train_accuracy': train_accuracy * 100})
      self.model.eval()
      with torch.no_grad():
        val_accuracy = 0
        count = 0
        for inputs, labels in self.dataloader_val:
            inputs = inputs.to(device)
            labels = labels.to(device)
            y_pred = self.model(inputs)
            val_loss = criterion(y_pred, labels)
            val_accuracy += (torch.argmax(y_pred, 1) == labels).float().sum()
            count += len(labels)
        val_accuracy /= count
        print("Epoch %d: model validation accuracy %.2f%%" % (epoch, val_accuracy*100))
        wandb.log({ 'epoch': epoch, 'validation_accuracy': val_accuracy * 100})
        wandb.log({ 'epoch': epoch, 'validation_loss': val_loss * 100})


In [ ]:
# sweep config file
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep - pretuning',
    'metric': {
      'goal': 'maximize',
      'name': 'validation_accuracy'
    },
    'parameters': {
        'lr': {
            'values': [1e-3, 1e-4]
        },
        'freeze_percent':{
            'values': [0.2, 0.6, 0.9]
        },
        'l2_reg':{
            'values': [0, 0.0005, 0.05]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'epochs':{
            'values': [5, 10]   #, 10]
        }
    }
}

In [ ]:
# Create a sweep
sweep_id = wandb.sweep(sweep = sweep_config, entity="abhinavtk", project='MA23M002-A2')

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 632bicm4
Sweep URL: https://wandb.ai/abhinavtk/MA23M002-A2/sweeps/632bicm4


In [ ]:
def main():
  with wandb.init() as run:
    run_name = "-bs_"+str(wandb.config.batch_size) +"-ep_"+str(wandb.config.epochs)+"-lr_"+str(wandb.config.lr) \
                  +"-freeze_"+str(wandb.config.freeze_percent)
    wandb.run.name = run_name
    pre_model = models.resnet50(pretrained=True)
    cnn_model = pretrainedCNN(trainset, valset, pre_model, batch_size=wandb.config.batch_size, freeze_percent = wandb.config.freeze_percent)
    cnn_model.train(wandb.config.epochs, wandb.config.lr, wandb.config.l2_reg)
   wandb.agent(sweep_id, function = main, count = 10)
wandb.finish()